In [ ]:
import pandas as pd
import requests
import json
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

In [ ]:
df = pd.read_csv("../data/df_baseline.csv")
test = pd.read_csv("../data/tests/diamonds_test.csv")

In [ ]:
df

In [ ]:
test

In [ ]:
df.describe()

In [ ]:
df = df[df['x'] > 0.05]
df = df[df['x'] < 90]

In [ ]:
df = df[df['y'] > 0.05]
df = df[df['y'] < 50]

In [ ]:
df = df[df['z'] > 2]


In [ ]:
df = df[df['depth'] > 50]
df = df[df['depth'] < 75]

In [ ]:
df = df[df['table'] > 45]
df = df[df['table'] < 75]

In [ ]:
df['L/W ratio'] = df['x'] / df['y']

In [ ]:
test.describe()

In [ ]:
test.loc[test['y'] < 2, 'y'] = 5.739648
test.loc[test['y'] > 10, 'y'] = 10

In [ ]:
test.loc[test['x'] < 2, 'x'] = 5.729978
test.loc[test['x'] > 10, 'x'] = 10

In [ ]:
test.loc[test['z'] < 2, 'z'] = 3.538479

In [ ]:
test.loc[test['table'] < 49, 'table'] = 57.490337
test.loc[test['table'] > 73, 'table'] = 57.490337

In [ ]:
test.loc[test['depth'] < 52, 'depth'] = 61.753581
test.loc[test['depth'] > 73, 'depth'] = 61.753581

In [ ]:
test['L/W ratio'] = test['x'] / test['y']
test

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
sns.jointplot(data=df, x="table", y="price")

In [ ]:
sns.jointplot(data=df, x="depth", y="price")

In [ ]:
sns.jointplot(data=df, x="x", y="price")

In [ ]:
sns.jointplot(data=df, x="y", y="price")

In [ ]:
sns.jointplot(data=df, x="carat", y="price")

In [ ]:
sns.jointplot(data=df, x="z", y="price")

In [ ]:
df.describe()

In [ ]:
test.describe()


In [ ]:
df

In [ ]:
test

In [ ]:
sns.histplot(data=test, x="carat")

In [ ]:
test.describe()

In [ ]:
test.describe()

## Categorical data

In [ ]:
#Extract City
df_cat = df[['cut','color','clarity']]
df_cat

In [ ]:
cols = df_cat.columns

In [ ]:
#Label encoding

le = LabelEncoder()
for i in cols:
    df_cat[i] = le.fit_transform(df_cat[i])
df_cat

In [ ]:
test_cols = ['cut','color','clarity']
for i in test_cols:
    test[i] = le.fit_transform(test[i])
test

# Numerical variables


In [ ]:
df_num = df[['price','carat','depth','table','x','y','z','L/W ratio']]
df_num

In [ ]:
test.describe()

In [ ]:
df_num.describe()

In [ ]:
df = pd.concat([df_num, df_cat], axis=1)

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
df = df[['price','carat','depth','table','L/W ratio','cut','color','clarity']]
df

In [ ]:
test = test[['carat','depth','table','L/W ratio','cut','color','clarity']]
test

In [ ]:
X = df[['carat','depth','table','L/W ratio','cut','color','clarity']]
y = df['price']

## Scaling

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
scaled_data

In [ ]:
scaled_df = pd.DataFrame(scaled_data, columns=['price','carat','depth','table','L/W ratio','cut','color','clarity'])
scaled_df.describe()

In [ ]:
scaled_test = scaler.fit_transform(test)
scaled_test

In [ ]:
scaled_test = pd.DataFrame(scaled_test, columns=['carat','depth','table','L/W ratio','cut','color','clarity'])
scaled_test.describe()

In [ ]:
X = scaled_df

In [ ]:
# MODEL

#model = RandomForestRegressor()
model = GradientBoostingRegressor()

In [ ]:
param_grid = {'n_estimators': [16, 32, 64, 128, 256, 512],
              'max_depth': [2, 4, 8, 16]}

In [ ]:
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)

In [ ]:
grid_search.fit(X,y)

In [ ]:
print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

In [ ]:
final_model = grid_search.best_estimator_

In [ ]:
#model = RandomForestRegressor(max_depth=16, n_estimators=128)
model = GradientBoostingRegressor(max_depth=8, n_estimators=256)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
check = pd.DataFrame({'Ground truth':y_test, 'Predictions':predictions, 'Diff':y_test-predictions})
check

In [ ]:
rmse = mean_squared_error(y_test, predictions, squared=False)
rmse

In [ ]:
model = RandomForestRegressor(max_depth=8, n_estimators=256)

In [ ]:
model.fit(X, y)

In [ ]:
predictions = model.predict(test)

In [ ]:
predictions

In [ ]:
result = pd.DataFrame({'id': range(0, len(predictions)), 'price': predictions})
result

In [ ]:
result.describe()

In [ ]:
result.to_csv("../submissions/submision_no_z_enlabel_random_forest.csv", index=False)

In [ ]:
X